title: "Detección de Mano con MediaPipe en Google Colab"
format: revealjs
---


## Detección de Mano con MediaPipe

En esta diapositiva se mostrará el código que captura una imagen desde la cámara, detecta la mano usando MediaPipe y dibuja los landmarks.


In [ ]:
# Importar las bibliotecas necesarias
import cv2
import numpy as np
import mediapipe as mp
from google.colab.output import eval_js
from IPython.display import display, Javascript, Image
from base64 import b64decode
import PIL
import io

# Función auxiliar para convertir una imagen de JavaScript a OpenCV
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Función para capturar una imagen usando la cámara del navegador en Google Colab
def take_photo(quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(t => t.stop());
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    img = js_to_image(data)
    return img

# Inicializar MediaPipe para la detección de manos
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Iniciar el modelo de detección de manos
with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7) as hands:
    # Tomar una foto
    img = take_photo()

    # Convertir la imagen de BGR a RGB para procesar con MediaPipe
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Procesar la imagen para detectar las manos
    results = hands.process(img_rgb)

    # Si se detecta una mano, dibujar los landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Convertir la imagen de BGR a RGB para mostrarla en Colab
    img_rgb_output = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_img = PIL.Image.fromarray(img_rgb_output)
    
    # Mostrar la imagen con los landmarks dibujados
    display(pil_img)

# 9. Conclusión

::: {.fragment .fade-in}
La **visión por computadora** está transformando la manera en que los jugadores interactúan con los videojuegos, permitiendo experiencias más inmersivas y accesibles. A medida que avanza la tecnología, veremos cada vez más innovaciones en la forma en que los jugadores controlan los juegos con gestos, movimientos y expresiones faciales.
:::

---

# 10. Referencias

1. MediaPipe Documentation: *Real-Time Hand and Face Detection*.
2. OpenCV Library: *Image Processing and Computer Vision*.
3. Kinect for Windows SDK: *Real-Time Body Motion Detection*.

---
